# MNIST Optuna Optimization

## Environment Setup

### Import Dependencies

In [1]:
import optuna
from optuna import Trial

import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger('optuna').setLevel(logging.WARNING)

import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../..')
# sys.path.insert(0, '../code/Users/f.chinnicarella/src/root_workspace/Bachelor-Thesis')

from utils.persistency.logger import Logger

from utils.dataset.build_dataset import load_MNIST_data
from utils.dataset.build_dataloader import init_data_loader

from utils.training.train_loop import full_train_loop
from utils.model.model_utils import init_model
from utils.optimization.early_stopper import EarlyStopper
from utils.optimization.regularizer import Regularizer
from utils.misc.device import get_device
from utils.model.model_utils import get_activation_fn, get_loss_fn, get_optimizer
from utils.optuna_utils.optuna_runner import OptunaRunner
from utils.optuna_utils.optuna_study_creator import OptunaStudyCreator
from utils.optuna_utils.pso_sampler import PSOSampler
from utils.display_results.display_results import prediction_loop
from utils.display_results.display_results import display_images
from utils.persistency.file_name_builder import file_name_builder, folder_exists_check

### Init Session

In [2]:
EXPERIMENT_NAME = 'MNIST_Optuna_Optimization'

In [3]:
SESSION_NUM = '013'

In [4]:
OUTPUTS_FOLDER_PATH_CSV = 'output_files_MNIST/csv'
OUTPUTS_FOLDER_PATH_TXT = 'output_files_MNIST/txt'
OUTPUTS_FOLDER_PATH_DB = 'output_files_MNIST/db'

## Load Data

In [5]:
train_dataset, val_dataset, test_dataset = load_MNIST_data('data_MNIST/')

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:09<00:00, 992615.17it/s] 


Extracting data_MNIST/MNIST\raw\train-images-idx3-ubyte.gz to data_MNIST/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 530642.91it/s]


Extracting data_MNIST/MNIST\raw\train-labels-idx1-ubyte.gz to data_MNIST/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:03<00:00, 540348.38it/s]


Extracting data_MNIST/MNIST\raw\t10k-images-idx3-ubyte.gz to data_MNIST/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<?, ?it/s]


Extracting data_MNIST/MNIST\raw\t10k-labels-idx1-ubyte.gz to data_MNIST/MNIST\raw



## Optuna Optimization

### Define Objective Function

In [6]:
def objective(trial: Trial, logger: Logger):
    # Define Hyperparameters - Structure HPs - Activation Function
    activation = trial.suggest_categorical('activation_fn', ['relu', 'sigmoid', 'tanh'])
    # activation = 'relu'
    
    # Define Hyperparameters - Structure HPs - Network Architecture (Depth)
    # num_hidden_layer = trial.suggest_int('num_hidden_layer', 3, 3)
    num_hidden_layer = 3

    # Define Hyperparameters - Structure HPs - Network Architecture (Width)
    network_architecture = [28 * 28]
    for i in range(num_hidden_layer):
        layer_width = trial.suggest_int(f'hidden_layer_n{i+1}_size', 0, 128, step=8)
        if layer_width != 0:
            network_architecture.append(layer_width)
    network_architecture.append(10)
    trial.set_user_attr('network', network_architecture)
    

    # Define Hyperparameters - Training HPs - Batch Size
    # batch_size = trial.suggest_int('batch_size', 16, 64, step=16)
    batch_size = 16
    
    # Define Hyperparameters - Training HPs - Learning Rate
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    # learning_rate = 1e-3
    
    # Define Hyperparameters - Training HPs - Loss Function
    # loss_function_str = trial.suggest_categorical('loss_fn', ['CrossEntropy', 'Focal'])
    loss_function_str = 'CrossEntropy'
    
    # Define Hyperparameters - Training HPs - Optimizer
    optimizer_str = trial.suggest_categorical('optimizer', ['SGD', 'Adam'])
    # optimizer_str = 'Adam'

    # Define Hyperparameters - Max Epochs
    max_epochs = 30


    # Init DataLoaders
    train_loader = init_data_loader(train_dataset, batch_size=batch_size)
    val_loader = init_data_loader(val_dataset, batch_size=batch_size)
    test_loader = init_data_loader(test_dataset, batch_size=batch_size)

    # Init Model
    model_extra_args = {'network_architecture': network_architecture, 'activation': get_activation_fn(activation)}
    model = init_model(model_str='MLP', extra_args=model_extra_args).to(get_device())

    # Init Loss
    loss_fn = get_loss_fn(loss_str=loss_function_str)
    
    # Init Optimizer
    optimizer = get_optimizer(model=model, optimizer_str=optimizer_str, learning_rate=learning_rate)

    # Init Regularizer
    regularizer = Regularizer(lambda_tot_widths=0.4, max_depth=3, max_width=128)

    # Init Early Stopper
    early_stopper = EarlyStopper(patience=5, mode="maximize")
    
    
    # Perform Training
    optim_score = full_train_loop(max_epochs=max_epochs,
                                  train_loader=train_loader, val_loader=val_loader, test_loader=test_loader,
                                  model=model,
                                  loss_fn=loss_fn,
                                  optimizer=optimizer,
                                  regularizer=regularizer,
                                  early_stopper=early_stopper,
                                  logger=logger,
                                  trial=trial)
    
    return optim_score

#### Optuna Constants - Study Parameters

In [7]:
ATTRS = ('number', 'value', 'user_attrs', 'state', 'params', 'duration', 'datetime_start', 'datetime_complete')

In [8]:
DIRECTION = 'maximize'

In [9]:
optuna_runner = OptunaRunner(objective_fn=objective,
                             n_jobs=-1,
                             n_trials=256,
                             path_csv=OUTPUTS_FOLDER_PATH_CSV,
                             path_txt=OUTPUTS_FOLDER_PATH_TXT,
                             session_num=SESSION_NUM,
                             metric_to_follow='accuracy',
                             attrs=ATTRS)

In [10]:
optuna_study_creator = OptunaStudyCreator(experiment_name=EXPERIMENT_NAME,
                                          path_db=OUTPUTS_FOLDER_PATH_DB, 
                                          session_num=SESSION_NUM,
                                          use_storage=True)

#### Optuna Constants - Samplers

In [ ]:
RandomSampler = optuna.samplers.RandomSampler()
TPESampler = optuna.samplers.TPESampler()
PSOSampler = PSOSampler(num_particles=32, max_generations=8)

#### Optuna Constants - Pruners

In [ ]:
MedianPruner = optuna.pruners.MedianPruner(n_startup_trials=0, n_warmup_steps=4, interval_steps=5, n_min_trials=4)
HyperbandPruner = optuna.pruners.HyperbandPruner(min_resource=3, max_resource=30, reduction_factor=3, bootstrap_count=6)

### Run Optimizations

#### Random Sampler

In [ ]:
study_name_Random = 'Random_Sampler'
study_Random = optuna_study_creator(study_name=study_name_Random, direction=DIRECTION, 
                                    sampler=RandomSampler, pruner=MedianPruner)
optuna_runner(study_Random, study_name_Random)

#### TPE Sampler

In [ ]:
study_name_TPE = 'TPE_Sampler'
study_TPE = optuna_study_creator(study_name=study_name_TPE, direction=DIRECTION, 
                                 sampler=TPESampler, pruner=HyperbandPruner)
optuna_runner(study_TPE, study_name_TPE)

#### PSO Sampler

In [ ]:
study_name_PSO = 'PSO_Sampler'
study_PSO = optuna_study_creator(study_name=study_name_PSO, direction=DIRECTION,
                                 sampler=PSOSampler, pruner=HyperbandPruner)
optuna_runner(study_PSO, study_name_PSO)

## Display Best Results ------[currently broken]

In [ ]:
# Alias for Best Study
best_study = study_TPE

In [ ]:
# Init DataLoaders
train_loader_best = init_data_loader(train_dataset, batch_size=best_study.best_params['batch_size'])
val_loader_best = init_data_loader(val_dataset, batch_size=best_study.best_params['batch_size'])
test_loader_best = init_data_loader(test_dataset, batch_size=best_study.best_params['batch_size'])

# Init Model
best_hidden_layer_sizes = [28*28] + [best_study.best_params[f'hidden_layer_n{i+1}_size'] for i in range(best_study.best_params['num_hidden_layer'])] + [10]
best_model_extra_args = {'network_architecture': best_hidden_layer_sizes,
                    'activation': get_activation_fn(best_study.best_params['activation_fn'])}
best_model = init_model(model_str='MLP', extra_args=best_model_extra_args).to(get_device())

# Init Loss and Optimizer
loss_fn_best = get_loss_fn(loss_str=best_study.best_params['loss_fn'])
optimizer_best = get_optimizer(model=best_model,
                               optimizer_str=best_study.best_params['optimizer'],
                               learning_rate=best_study.best_params['learning_rate'])

# Init Early Stopper
early_stopper_best = EarlyStopper(patience=5, mode="maximize")

# Init Logger
folder_exists_check(OUTPUTS_FOLDER_PATH_TXT, SESSION_NUM, f'log_BEST_STUDY')
logger_best_study = Logger(file_name_builder(OUTPUTS_FOLDER_PATH_TXT, SESSION_NUM, f'log_BEST_STUDY', 'txt'))


# Train Model
full_train_loop(max_epochs=best_study.best_params['max_epochs'],
                train_loader=train_loader_best, val_loader=val_loader_best, test_loader=test_loader_best,
                model=best_model,
                loss_fn=loss_fn_best,
                optimizer=optimizer_best,
                regularizer=None,
                early_stopper=early_stopper_best,
                logger=logger_best_study,
                trial=None)

In [ ]:
images, y_pred_best, y_test = prediction_loop(test_loader_best, best_model)

In [ ]:
display_images(images, y_pred_best, y_test, 50)

## DB Operations

### Copy Study to from other DB

In [11]:
optuna_study_creator.copy_study_into_new_db('TPE_Sampler', "012", EXPERIMENT_NAME + '_temp')

### Delete Study from DB

In [ ]:
raise Exception('Delete Study from DB')
# study_name_to_delete = ''
session_to_delete = '012'
optuna_study_creator.delete_study_from_db(study_name_to_delete, session_to_delete)

### Load Unfinished Study from DB

In [ ]:
unfinished_study = optuna_study_creator.load_study_from_db(study_name_Random, SESSION_NUM)